# Detecção de Bots no Twitter Utilizando BERT e Keras

Neste projeto, vamos treinar um modelo BERT para detecção de bots no Twitter utilizando o dataset Twitter-Bot Detection do Kaggle. Todo o código será executado no Google Colab, e o dataset será baixado usando a biblioteca gdown. Além disso, utilizaremos a biblioteca Plotly para visualização de gráficos durante o processo de análise e avaliação do modelo.

## 1. Instalação das Bibliotecas Necessárias

Primeiro, precisamos instalar as bibliotecas que serão usadas: transformers, tensorflow, gdown e plotly.

In [ ]:
!pip install scikit-learn torch

In [17]:
!pip install transformers tensorflow gdown plotly

### 1.1. Importação de Bibliotecas


In [ ]:
import gdown
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import plotly.express as px
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

## 2. Download do Dataset

Vamos utilizar o gdown para baixar o dataset do Kaggle diretamente para o ambiente do Google Colab.

In [ ]:
arquivo_destino_colab = "dataset.csv"
doc_id = "14ZeKj-r84KcTQ_O9iYSFoteEZsYKPsbz"
URL = f"https://drive.google.com/uc?id={doc_id}"
gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=14ZeKj-r84KcTQ_O9iYSFoteEZsYKPsbz
To: /content/dataset.csv
100%|██████████| 7.46M/7.46M [00:00<00:00, 223MB/s]


'dataset.csv'

In [ ]:
dados = pd.read_csv(arquivo_destino_colab)
dados

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention
...,...,...,...,...,...,...,...,...,...,...,...
49995,491196,uberg,Want but put card direction know miss former h...,64,0,9911,True,1,Lake Kimberlyburgh,2023-04-20 11:06:26,teach quality ten education any
49996,739297,jessicamunoz,Provide whole maybe agree church respond most ...,18,5,9900,False,1,Greenbury,2022-10-18 03:57:35,add walk among believe
49997,674475,lynncunningham,Bring different everyone international capital...,43,3,6313,True,1,Deborahfort,2020-07-08 03:54:08,onto admit artist first
49998,167081,richardthompson,Than about single generation itself seek sell ...,45,1,6343,False,0,Stephenside,2022-03-22 12:13:44,star


In [ ]:
# Exibindo as colunas e o tipo de dado de cada uma
print("\nInformações gerais sobre o dataset:")
print(dados.info())


Informações gerais sobre o dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   User ID         50000 non-null  int64 
 1   Username        50000 non-null  object
 2   Tweet           50000 non-null  object
 3   Retweet Count   50000 non-null  int64 
 4   Mention Count   50000 non-null  int64 
 5   Follower Count  50000 non-null  int64 
 6   Verified        50000 non-null  bool  
 7   Bot Label       50000 non-null  int64 
 8   Location        50000 non-null  object
 9   Created At      50000 non-null  object
 10  Hashtags        41659 non-null  object
dtypes: bool(1), int64(5), object(5)
memory usage: 3.9+ MB
None


In [ ]:
# Verificando a presença de valores nulos
print("\nVerificando valores ausentes:")
display(dados.isnull().sum())


Verificando valores ausentes:


,0
User ID,0
Username,0
Tweet,0
Retweet Count,0
Mention Count,0
Follower Count,0
Verified,0
Bot Label,0
Location,0
Created At,0


In [18]:
# Exibindo estatísticas descritivas dos dados numéricos
print("\nEstatísticas descritivas:")
display(dados.describe())


Estatísticas descritivas:


,User ID,Retweet Count,Mention Count,Follower Count,Bot Label
count,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000
mean,548890.680540,50.00560,2.513760,4988.602380,0.500360
std,259756.681425,29.18116,1.708563,2878.742898,0.500005
min,100025.000000,0.00000,0.000000,0.000000,0.000000
25%,323524.250000,25.00000,1.000000,2487.750000,0.000000
50%,548147.000000,50.00000,3.000000,4991.500000,1.000000
75%,772983.000000,75.00000,4.000000,7471.000000,1.000000
max,999995.000000,100.00000,5.000000,10000.000000,1.000000


## (EXTRA) Tratando dados nulos

In [ ]:
# Verificando a porcentagem de valores nulos na coluna 'Hashtags'
percentual_nulos = dados['Hashtags'].isnull().mean() * 100
print(f'Percentual de valores nulos na coluna Hashtags: {percentual_nulos:.2f}%')


Percentual de valores nulos na coluna Hashtags: 16.68%


In [ ]:
# Dividir as frases em palavras, considerando cada palavra como uma "hashtag"
dados['Hashtags_extracted'] = dados['Hashtags'].dropna().apply(lambda x: x.split())

# Exibir as primeiras 10 células de hashtags extraídas
print(dados['Hashtags_extracted'].head(10))


0                                        NaN
1                               [both, live]
2                             [phone, ahead]
3                    [ever, quickly, new, I]
4                         [foreign, mention]
5    [anyone, respond, perhaps, market, run]
6                                [president]
7                   [option, husband, admit]
8                                        NaN
9                         [available, thing]
Name: Hashtags_extracted, dtype: object


In [ ]:
# Explodir a lista de palavras (tratadas como hashtags)
hashtags_planas = [hashtag for sublist in dados['Hashtags_extracted'].dropna() for hashtag in sublist]

# Obter valores únicos
valores_unicos_hashtags = pd.Series(hashtags_planas).unique()

print(valores_unicos_hashtags)


['both' 'live' 'phone' 'ahead' 'ever' 'quickly' 'new' 'I' 'foreign'
 'mention' 'anyone' 'respond' 'perhaps' 'market' 'run' 'president'
 'option' 'husband' 'admit' 'available' 'thing' 'treat' 'care' 'eat'
 'author' 'upon' 'all' 'direction' 'fly' 'course' 'difficult' 'fine'
 'today' 'question' 'about' 'tough' 'interesting' 'every' 'cost' 'form'
 'foot' 'year' 'control' 'health' 'do' 'fact' 'particularly' 'their'
 'plant' 'account' 'wonder' 'purpose' 'position' 'able' 'have' 'that'
 'especially' 'effort' 'situation' 'others' 'different' 'sort' 'small'
 'knowledge' 'mission' 'owner' 'know' 'budget' 'reason' 'significant' 'at'
 'take' 'sense' 'sell' 'officer' 'rate' 'board' 'front' 'before' 'enter'
 'cover' 'gun' 'series' 'instead' 'behind' 'response' 'learn' 'design'
 'energy' 'begin' 'former' 'impact' 'above' 'most' 'model' 'mind'
 'sometimes' 'nature' 'green' 'story' 'teacher' 'late' 'land' 'discover'
 'find' 'determine' 'sure' 'parent' 'old' 'medical' 'treatment' 'as'
 'center' 'on' 'ma

In [ ]:
# Preencher valores nulos com uma lista vazia
dados['Hashtags'] = dados['Hashtags'].fillna("Sem Hashtags")

In [ ]:
# Selecionar apenas colunas numéricas
dados_numericos = dados.select_dtypes(include=['float64', 'int64'])
# Calcular a matriz de correlação
matriz_correlacao = dados_numericos.corr()
# Criar um heatmap com Plotly Express
fig = px.imshow(matriz_correlacao,
                text_auto=True,  # Mostrar os valores no gráfico
                color_continuous_scale='RdBu_r',  # Escolher a escala de cores
                title="Matriz de Correlação")

# Exibir o gráfico
fig.show()

## Preparação dos Dados
Agora vamos preparar a coluna que contém os rótulos de Bots e Não Bots para o treinamento.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Supondo que o dataset tem uma coluna 'Tweet' e uma coluna 'Bot Label'
X = dados['Tweet']
y = dados['Bot Label']  # Onde 1 é bot, 0 é não bot

# Dividir os dados em treino e teste
train_texts, test_texts, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=42)


## Tokenização dos dados usando o tokenizador BERT



O BERT requer que os textos sejam tokenizados corretamente. Vamos usar o BertTokenizer para isso.

In [ ]:
# Tokenização utilizando BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizando os textos de treino e teste
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



## Treinando o modelo
Agora, vamos carregar o modelo BERT para classificação de sequência.

In [ ]:
# Criar dataset customizado para o treinamento
class BotDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Criar datasets de treino e teste
train_dataset = BotDataset(train_encodings, y_train.tolist())
test_dataset = BotDataset(test_encodings, y_test.tolist())

# Carregar o modelo BERT para classificação
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Definir otimizador e taxa de aprendizado
optimizer = AdamW(model.parameters(), lr=5e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



In [ ]:
import torch
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

# Avaliação do modelo (após o treinamento)
model.eval()
eval_loss = 0
eval_accuracy = 0
nb_eval_steps = 0
all_labels = []
all_predictions = []
all_probs = []  # Para calcular AUC-ROC

# Loop sobre o dataset de teste
for batch in test_dataset:
    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=batch['input_ids'].unsqueeze(0),
                        attention_mask=batch['attention_mask'].unsqueeze(0),
                        labels=batch['labels'].unsqueeze(0))

    loss = outputs.loss
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)  # Converter logits em probabilidades

    # Calcular a acurácia
    predictions = torch.argmax(logits, dim=-1)
    eval_accuracy += (predictions == batch['labels']).sum().item()

    # Armazenar previsões e rótulos para calcular métricas adicionais
    all_labels.extend(batch['labels'].cpu().numpy().flatten())  # Garantir que labels sejam 1-D
    all_predictions.extend(predictions.cpu().numpy().flatten())  # Garantir que as previsões sejam 1-D
    all_probs.extend(probs[:, 1].cpu().numpy().flatten())  # Probabilidade da classe positiva (para AUC-ROC)

    eval_loss += loss.item()
    nb_eval_steps += 1

# Exibir métricas finais
accuracy = eval_accuracy / len(test_dataset)
avg_loss = eval_loss / nb_eval_steps

# Cálculo das métricas usando os resultados armazenados
f1 = f1_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
roc_auc = roc_auc_score(all_labels, all_probs)

print(f"Acurácia do modelo: {accuracy:.2f}")
print(f"Perda (Loss): {avg_loss:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Precisão (Precision): {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

## Conclusão (Pontos de Melhoria) (Extra)

### Obter Mais Memória (RAM e GPU):

- Capacidade de Memória: O modelo está demorando muito para treinar e/ou estou encontrando erros de falta de memória. Treinamento de modelos grandes como BERT ou outros transformers requerem uma quantidade significativa de memória, tanto RAM quanto memória de GPU. Aumentar a memória disponível pode reduzir o tempo de treinamento e permitir o uso de datasets maiores.
- Utilização de GPUs: Treinamento de modelos de deep learning como BERT em GPUs (ou até TPUs) pode acelerar significativamente o processo. Por está limitado por uma CPU, considere a migração para GPUs, que são muito mais adequadas para computações paralelas intensas.


### Experimentar Diferentes Hiperparâmetros:
- Ajuste de Taxa de Aprendizado: A taxa de aprendizado (learning rate) tem um grande impacto no desempenho do modelo. Testar diferentes valores pode ajudar a encontrar o equilíbrio entre convergência rápida e evitar overfitting.

- Número de Épocas: Dependendo do comportamento da curva de aprendizado, ajustar o número de épocas pode melhorar a performance do modelo. Menos épocas podem evitar overfitting, enquanto mais épocas podem ajudar se o modelo estiver subajustado.

###Utilização de Técnicas de Regularização:
- Dropout: Se o modelo parecer estar sobreajustando os dados de treinamento, você pode aumentar o valor da taxa de dropout para adicionar mais regularização. O Dropout ajuda a reduzir o overfitting, forçando o modelo a aprender representações mais robustas.
- L2 Regularization: A adição de regularização L2 ao otimizador pode ajudar a penalizar pesos muito grandes no modelo, melhorando sua capacidade de generalizar para novos dados.